In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(['Unnamed: 32','id'],axis=1)
x = df.drop('diagnosis',axis=1)

In [ ]:
x.head()

In [ ]:
df = pd.get_dummies(df,drop_first=True)
df.head()

In [ ]:
y = df['diagnosis_M']
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
RL = LogisticRegression(random_state=0).fit(X_train,y_train)

In [ ]:
rl_pred = RL.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn import tree
tree = tree.DecisionTreeClassifier(random_state=0).fit(X_train,y_train)

In [ ]:
predtree = tree.predict(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(X_train,y_train)

In [ ]:
predKNN = KNN.predict(X_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
NB = GaussianNB().fit(X_train,y_train)

In [ ]:
ynb = NB.predict(X_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ADA = AdaBoostClassifier(random_state=0).fit(X_train,y_train)

In [ ]:
yada = ADA.predict(X_test)

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bag = BaggingClassifier(random_state=0).fit(X_train,y_train)

In [ ]:
ybag = bag.predict(X_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
for i,model in enumerate([RL,NB,tree,ADA,bag]):
    cm = confusion_matrix(y_test, model.predict(X_test))
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(i)
    sns.heatmap(cm, annot=True, annot_kws={"size":10}, 
            cmap='Blues', square=True, fmt='.3f')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion matrix for:\n{}'.format(model.__class__.__name__));

In [ ]:
]from sklearn.metrics import fbeta_score, accuracy_score
from time import time

'''
     entradas:
        - learner: el algoritmo de aprendizaje que se va a entrenar y predecir
        - sample_size: el tamaño de las muestras (número) que se extraerán del conjunto de entrenamiento
        - X_train: conjunto de entrenamiento de características
        - y_train: conjunto de entrenamiento de ingresos
        - X_test: conjunto de pruebas de características
        - y_test: conjunto de pruebas de ingresos
     '''

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test):
    results = {}
    
    # Ajustar al alumno a los datos de entrenamiento usando cortes con 'sample_size'
    start = time() # Obtener hora de inicio
    learner = learner.fit(X_train[:sample_size],y_train[:sample_size])
    end = time() # Obtener hora de termino
    
    # Calcular el tiempo de entrenamiento
    results['train_time'] = end - start
        
    #  Get the predictions on the test set,
    #  then get predictions on the first 300 training samples
    start = time() # Obtener hora de inicio
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time() # Obtener hora de termino
    
   # Calcular el tiempo total de predicción
    results['pred_time'] = end - start
            
    # Calcular la precisión en las primeras 300 muestras de entrenamiento
    results['acc_train'] = accuracy_score(y_train[:300],predictions_train)
        
   # Calcular la precisión en el conjunto de prueba
    results['acc_test'] = accuracy_score(y_test,predictions_test)
    
    # Calcule la puntuación F en las primeras 300 muestras de entrenamiento
    results['f_train'] = fbeta_score(y_train[:300],predictions_train,beta=0.5)
        
    # Calcule la puntuación F en el conjunto de prueba
    results['f_test'] = fbeta_score(y_test,predictions_test,beta=0.5)
       
    # Éxito
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

In [ ]:

samples_1 = int(round(len(X_train) / 100))
samples_10 = int(round(len(X_train) / 10))
samples_100 = len(X_train)
results = {}
for clf in [RL,NB,tree,ADA,bag]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

In [ ]:
for i in results.items():
    print(i[0])
    display(pd.DataFrame(i[1]).rename(columns={0:'1%', 1:'10%', 2:'100%'}))